<div style="text-align: center; padding-top: 30px; padding-bottom: 10px;">

<h1 style="font-size: 2.8em; font-weight: 600; margin-bottom: 0.2em;">
Monte Carlo simulation
</h1>

<p style="font-size: 1.2em; color: gray; font-style: italic; margin-top: 0;">
This notebook visualises the main results from the paper.
</p>

</div>

In [1]:
from simulations.simulation_functions import simulate, Surface
import numpy as np
import plotly.graph_objects as go
from utils import create_pred_input
from pathlib import Path


2026-02-10 11:48:01.589968: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-10 11:48:01.736640: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-10 11:48:01.949047: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-10 11:48:02.318778: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770724083.025354   33868 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770724083.08

In [8]:
#function to calculate the surface for the model, benchmark or bias, depending on the element argument. 
################################################## User Input ##################################################
specification = "linear" # "linear", "Burke", "Leirvik", "nonlinear" - true model to be estimated
element = "bias" # "model", "benchmark", "bias" - element to be plotted
model = "NN" # "NN" or "Quadratic" - model that is thrying to estimate the true model

###############################################################################################################

def calculate_surface(specification, element, model):
    data= simulate(seed=0, n_countries=196, n_years=63, specification=specification, add_noise=True, sample_data=False, dynamic=False)
    _, T, P = create_pred_input(
        True,
        mean_T=np.mean(data["temperature"]),
        std_T=np.std(data["temperature"]),
        mean_P=np.mean(data["precipitation"]),
        std_P=np.std(data["precipitation"])
    )
    
    
   
    if model=="NN":
        
        folder = Path(f"../results/paper/weights/monte_carlo/NN/{specification}")
        
        surface = np.load(next(folder.glob("*.npy")))   # raises StopIteration if folder empty
    else:
        folder = Path(f"../results/paper/weights/monte_carlo/Quadratic/{specification}")
        surface = np.load(next(folder.glob("*.npy")))   # raises StopIteration if folder empty
        
    model_values = np.mean(surface, axis=0).reshape(T.shape)
    benchmark = Surface(T.flatten(), P.flatten(), specification=specification, dynamic=False, time_periods=63).reshape(T.shape)
    

    if element == 'model':
    
        surf = go.Surface(
            x=T, y=P, z=model_values,
            colorscale='Plasma',
            opacity=0.85,
            showscale=False,
            name=f'Model'
            
        )

    elif element == 'benchmark':
        surf= go.Surface(
            x=T, y=P, z=benchmark,
            colorscale='Plasma',
            opacity=0.85,
            showscale=False,
            name=f'Benchmark'
        )
    
    else: 
        bias = np.array(model_values - benchmark)

        zmin, zmax = 0, 1
        ticks = np.linspace(zmin, zmax, 9)           
        ticktext = [f"{t:.1f}" for t in ticks]       

        surf = go.Heatmap(
            x=P[:, 0], y=T[0, :], z=np.sqrt(np.square(bias.T)),
            colorscale='ylorrd', zmin=zmin, zmax=zmax,
            showscale=True,
            colorbar=dict(
                title='Bias',
                thickness=15,
                len=1,
                tickmode='array',
                tickvals=ticks,
                ticktext=ticktext,
            )
        )
    fig=create_figure(element, surf)
    
  
    if element=="bias":
        return fig, bias
    else:
        return fig


# creates specifc layout for each element type (model, benchmark, bias)
def create_figure(element, plot_data):
    
    if element=="model" or element=="benchmark":
        fig = go.Figure(data=plot_data)
        fig.update_layout(
               autosize=False,
        width=500,
        height=600,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4,
        ),

            scene=dict(
                xaxis_title='Temperature (°C)',
                yaxis_title='Precipitation (m)',
                zaxis=dict(autorange=False, title='Δ ln(Growth)', range=[-0.6, 0.2]),
                camera=dict(eye=dict(x=2.35, y=0.006, z=0.4))
            ),
            
            legend=dict(
                bgcolor='rgba(255,255,255,0.7)',
                bordercolor='black',
                borderwidth=1
            ),
            font=dict(
            size=10
        )
        )
        
    else: 
        layout = go.Layout(
        yaxis=dict(autorange='reversed'),
        font=dict(
            size=16
        ))
        fig = go.Figure(data=plot_data, layout=layout)
        fig.update_layout(
        xaxis=dict(title='Precipitation (m)'),
        yaxis=dict(title='Temperature (°C)')
    )



    return fig


In [11]:
fig =calculate_surface("Burke", "model", "NN")

fig.show()
